# Assignment 6: Neural Network Showdown

Build and compare neural network architectures on image and time-series data.

## Setup

In [2]:
%pip install -q -r requirements.txt

# GPU acceleration (platform-specific)
import platform
if platform.system() == "Darwin" and platform.machine() == "arm64":
    %pip install -q tensorflow-metal

%reset -f

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
import numpy as np
import pandas as pd

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
tf.random.set_seed(42)
np.random.seed(42)

# Report available accelerators
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU acceleration: {len(gpus)} device(s)")
    for gpu in gpus:
        print(f"  {gpu.name}")
else:
    print("No GPU detected — using CPU")

from tensorflow import keras
from keras import Sequential
from keras.layers import (
    Dense, Flatten, Dropout, Conv2D, MaxPooling2D, LSTM, Input
)
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix

from helpers import (
    load_cifar10, load_ecg5000,
    plot_training_history, plot_confusion_matrix,
    plot_sample_images, plot_ecg_traces, plot_predictions,
    CIFAR10_CLASSES, ECG_CLASSES,
)

OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Setup complete!")

GPU acceleration: 1 device(s)
  /physical_device:GPU:0
Setup complete!


---

## Part 1: Dense Baseline on CIFAR-10

**Task:** Build a Dense (fully connected) network to classify CIFAR-10 images.

CIFAR-10 has 60,000 color images (32x32x3) across 10 classes. A Dense network
flattens each image into 3,072 numbers and classifies from there. This is our
baseline — it ignores spatial structure entirely.

**Architecture requirements:**
- Flatten the input
- At least 2 hidden Dense layers with ReLU activation
- Dropout after each hidden layer
- Output: Dense(10, softmax)

In [4]:
print("Part 1: Dense Baseline on CIFAR-10")
print("-" * 40)

# Load data (normalized to [0,1], one-hot encoded)
X_train, y_train, X_test, y_test = load_cifar10()
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

Part 1: Dense Baseline on CIFAR-10
----------------------------------------
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step
Train: (50000, 32, 32, 3), Test: (10000, 32, 32, 3)


In [5]:
# Visualize some training images to verify data loaded correctly
plot_sample_images(X_train, y_train, CIFAR10_CLASSES)

/Users/choisinu/Desktop/UCSF-DSC223 WINTER26 /WEEK6/06-brainiac-Choisinu9999/helpers.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [6]:
# TODO: Build a Dense model using Sequential
# Tip: call model_dense.summary() after building to verify your architecture
# Requirements:
#   - Input(shape=(32, 32, 3))
#   - Flatten()
#   - At least 2 Dense hidden layers with activation='relu'
#   - Dropout after each hidden Dense layer
#   - Dense(10, activation='softmax') as output
model_dense = model_dense = Sequential([
    Input(shape=(32, 32, 3)),
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(10, activation="softmax"),
])

model_dense.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       786,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 820,874 (3.13 MB)

 Trainable params: 820,874 (3.13 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# TODO: Compile the model
# model_dense.compile(optimizer='adam',
#                     loss='categorical_crossentropy',
#                     metrics=['accuracy'])

model_dense.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [8]:
# TODO: Train with EarlyStopping
# early_stop = EarlyStopping(monitor='val_loss', patience=3,
#                            restore_best_weights=True)
# history_dense = model_dense.fit(X_train, y_train,
#                                 epochs=20, batch_size=128,
#                                 validation_split=0.1,
#                                 callbacks=[early_stop])
history_dense = early_stop = EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

history_dense = model_dense.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128,
    validation_split=0.1,
    callbacks=[early_stop],
) # replace with your fit call

Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 35s 64ms/step - accuracy: 0.1223 - loss: 19.8769 - val_accuracy: 0.2166 - val_loss: 3.6504
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 17s 49ms/step - accuracy: 0.1460 - loss: 5.6958 - val_accuracy: 0.1474 - val_loss: 2.2836
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 14s 39ms/step - accuracy: 0.1556 - loss: 2.3205 - val_accuracy: 0.1920 - val_loss: 2.1360
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - accuracy: 0.1715 - loss: 2.2365 - val_accuracy: 0.2122 - val_loss: 2.0649
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 37s 105ms/step - accuracy: 0.1927 - loss: 2.1872 - val_accuracy: 0.2214 - val_loss: 2.0575
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.2148 - loss: 2.1361 - val_accuracy: 0.2626 - val_loss: 1.9990
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 24s 69ms/step - accuracy: 0.2355 - loss: 2.1032 - val_accuracy: 0.2722 - val_loss: 1.9785
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 16s 44ms/step - accuracy: 0.2432 - loss: 2.0954 

In [9]:
# TODO: Evaluate on test set
# test_loss, test_acc = model_dense.evaluate(X_test, y_test, verbose=0)
test_loss, test_acc = model_dense.evaluate(X_test, y_test, verbose=0) # replace

# TODO: Generate predictions and confusion matrix
# y_pred = np.argmax(model_dense.predict(X_test, verbose=0), axis=1)
# y_true = np.argmax(y_test, axis=1)
# cm = confusion_matrix(y_true, y_pred)
y_pred = np.argmax(model_dense.predict(X_test, verbose=0), axis=1)
y_true = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true, y_pred)

In [10]:
# Optional: visualize predictions and confusion matrix to diagnose issues
# plot_predictions(X_test, y_true, y_pred, CIFAR10_CLASSES)
# plot_confusion_matrix(y_true, y_pred, list(CIFAR10_CLASSES.values()),
#                       os.path.join(OUTPUT_DIR, "part1_confusion_matrix.png"))

In [11]:
# Save results (do not modify this cell)
results = {
    "accuracy": float(test_acc),
    "confusion_matrix": cm.tolist(),
}
with open(os.path.join(OUTPUT_DIR, "part1_results.json"), "w") as f:
    json.dump(results, f, indent=2)

print(f"Dense accuracy: {test_acc:.4f}")
print("Saved output/part1_results.json")

Dense accuracy: 0.2782
Saved output/part1_results.json


---

## Part 2: CNN on CIFAR-10

**Task:** Build a CNN to classify the same CIFAR-10 images. Compare its accuracy
to the Dense baseline from Part 1.

CNNs use convolutional filters that slide across the image, detecting local
patterns (edges, textures, shapes). This preserves spatial structure that
Dense layers discard.

**Architecture requirements:**
- At least 2 Conv2D + MaxPooling2D blocks
- Flatten, then Dense hidden layer with Dropout
- Output: Dense(10, softmax)

In [12]:
print("\nPart 2: CNN on CIFAR-10")
print("-" * 40)

# Data is already loaded from Part 1 (X_train, y_train, X_test, y_test)


Part 2: CNN on CIFAR-10
----------------------------------------


In [13]:
# TODO: Build a CNN model using Sequential
# Requirements:
#   - Input(shape=(32, 32, 3))
#   - At least 2 blocks of: Conv2D(filters, (3,3), activation='relu')
#                            + MaxPooling2D((2,2))
#   - Flatten()
#   - Dense hidden layer with ReLU + Dropout
#   - Dense(10, activation='softmax') as output
# Tip: call model_cnn.summary() after building to check layer shapes and param counts
model_cnn = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(32, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dense(10, activation="softmax"),
])

model_cnn.summary() # replace with your Sequential model

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 167,562 (654.54 KB)

 Trainable params: 167,562 (654.54 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# TODO: Compile the model
# model_cnn.compile(optimizer='adam',
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])

model_cnn.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [15]:
# TODO: Train with EarlyStopping and ModelCheckpoint
# callbacks = [
#     EarlyStopping(monitor='val_loss', patience=3,
#                   restore_best_weights=True),
#     ModelCheckpoint('output/best_cnn.keras',
#                     save_best_only=True, monitor='val_accuracy'),
# ]
# history_cnn = model_cnn.fit(X_train, y_train,
#                             epochs=15, batch_size=64,
#                             validation_split=0.1,
#                             callbacks=callbacks)

callbacks = [
    EarlyStopping(
        monitor="val_loss", patience=3, restore_best_weights=True
    ),
    ModelCheckpoint(
        "output/best_cnn.keras",
        save_best_only=True,
        monitor="val_accuracy",
    ),
]
history_cnn = model_cnn.fit(
    X_train, y_train,
    epochs=15,
    batch_size=64,
    validation_split=0.1,
    callbacks=callbacks,
)

Epoch 1/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 87s 105ms/step - accuracy: 0.4115 - loss: 1.6216 - val_accuracy: 0.4938 - val_loss: 1.4134
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 60s 85ms/step - accuracy: 0.5216 - loss: 1.3426 - val_accuracy: 0.5654 - val_loss: 1.2452
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 58s 83ms/step - accuracy: 0.5461 - loss: 1.3062 - val_accuracy: 0.5428 - val_loss: 1.3276
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 54s 77ms/step - accuracy: 0.5233 - loss: 1.4349 - val_accuracy: 0.5886 - val_loss: 1.2611
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 54s 76ms/step - accuracy: 0.4761 - loss: 1.8399 - val_accuracy: 0.4470 - val_loss: 2.0735


In [16]:
# TODO: Plot training history
# plot_training_history(history_cnn,
#                       os.path.join(OUTPUT_DIR, "part2_training_history.png"))

plot_training_history(
    history_cnn, os.path.join(OUTPUT_DIR, "part2_training_history.png")
)

In [17]:
# TODO: Evaluate on test set
# cnn_loss, cnn_acc = model_cnn.evaluate(X_test, y_test, verbose=0)
cnn_loss, cnn_acc = model_cnn.evaluate(X_test, y_test, verbose=0)


# TODO: Generate predictions and confusion matrix
# y_pred_cnn = np.argmax(model_cnn.predict(X_test, verbose=0), axis=1)
# y_true_cnn = np.argmax(y_test, axis=1)
# cm_cnn = confusion_matrix(y_true_cnn, y_pred_cnn)
y_pred_cnn = np.argmax(model_cnn.predict(X_test, verbose=0), axis=1)
y_true_cnn = np.argmax(y_test, axis=1)
cm_cnn = confusion_matrix(y_true_cnn, y_pred_cnn)

In [18]:
# Optional: visualize predictions and confusion matrix to diagnose issues
# plot_predictions(X_test, y_true_cnn, y_pred_cnn, CIFAR10_CLASSES)
# plot_confusion_matrix(y_true_cnn, y_pred_cnn, list(CIFAR10_CLASSES.values()),
#                       os.path.join(OUTPUT_DIR, "part2_confusion_matrix.png"))

In [19]:
# Save results (do not modify this cell)
results_cnn = {
    "accuracy": float(cnn_acc),
    "confusion_matrix": cm_cnn.tolist(),
}
with open(os.path.join(OUTPUT_DIR, "part2_results.json"), "w") as f:
    json.dump(results_cnn, f, indent=2)

comparison = pd.DataFrame([
    {"model": "Dense", "accuracy": float(test_acc)},
    {"model": "CNN", "accuracy": float(cnn_acc)},
])
comparison.to_csv(os.path.join(OUTPUT_DIR, "part2_comparison.csv"), index=False)

print(f"CNN accuracy:   {cnn_acc:.4f}")
print(f"Dense accuracy: {test_acc:.4f}")
print(f"Improvement:    {cnn_acc - test_acc:+.4f}")
print("Saved output/part2_results.json and output/part2_comparison.csv")

CNN accuracy:   0.5501
Dense accuracy: 0.2782
Improvement:    +0.2719
Saved output/part2_results.json and output/part2_comparison.csv


---

## Part 3: LSTM on ECG5000

**Task:** Build an LSTM to classify heartbeat recordings.

ECG5000 contains 5,000 heartbeat recordings — each is 140 time steps of voltage
measurements, classified into 5 types (Normal, Supraventricular, Premature
Ventricular, Fusion, Unknown). This is sequential data where order matters,
making it a natural fit for recurrent networks.

**Architecture requirements:**
- LSTM layer (any reasonable number of units)
- Dropout for regularization
- Dense output with softmax (5 classes)

In [20]:
print("\nPart 3: LSTM on ECG5000")
print("-" * 40)

# Load ECG data (already shaped for RNN input)
X_train_ecg, y_train_ecg, X_test_ecg, y_test_ecg = load_ecg5000()
print(f"Train: {X_train_ecg.shape}, Test: {X_test_ecg.shape}")
print(f"Classes: {list(ECG_CLASSES.values())}")


Part 3: LSTM on ECG5000
----------------------------------------
Train: (4000, 140, 1), Test: (1000, 140, 1)
Classes: ['Normal', 'Supraventricular', 'Premature Ventricular', 'Fusion', 'Unknown']


In [21]:
# Visualize ECG traces to understand the data
plot_ecg_traces(X_train_ecg, y_train_ecg, ECG_CLASSES)

/Users/choisinu/Desktop/UCSF-DSC223 WINTER26 /WEEK6/06-brainiac-Choisinu9999/helpers.py:211: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [22]:
# TODO: Build an LSTM model using Sequential
# Requirements:
#   - Input(shape=(140, 1))
#   - LSTM layer (e.g., 64 units)
#   - Dropout
#   - Dense(5, activation='softmax')
# Tip: call model_lstm.summary() after building to verify your architecture
model_lstm = Sequential([
    Input(shape=(140, 1)),
    LSTM(64),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(5, activation="softmax"),
])

model_lstm.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,141 (74.77 KB)

 Trainable params: 19,141 (74.77 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# TODO: Compile the model
# model_lstm.compile(optimizer='adam',
#                    loss='categorical_crossentropy',
#                    metrics=['accuracy'])

model_lstm.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [24]:
# TODO: Train with EarlyStopping
# early_stop = EarlyStopping(monitor='val_loss', patience=5,
#                            restore_best_weights=True)
# history_lstm = model_lstm.fit(X_train_ecg, y_train_ecg,
#                               epochs=30, batch_size=32,
#                               validation_split=0.1,
#                               callbacks=[early_stop])
early_stop = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

history_lstm = model_lstm.fit(
    X_train_ecg, y_train_ecg,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
)  # replace with your fit call

Epoch 1/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 17s 83ms/step - accuracy: 0.8742 - loss: 0.5575 - val_accuracy: 0.8925 - val_loss: 0.3989
Epoch 2/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9253 - loss: 0.2789 - val_accuracy: 0.9125 - val_loss: 0.3305
Epoch 3/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.9372 - loss: 0.2398 - val_accuracy: 0.9150 - val_loss: 0.3307
Epoch 4/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9383 - loss: 0.2306 - val_accuracy: 0.9100 - val_loss: 0.3017
Epoch 5/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9428 - loss: 0.2166 - val_accuracy: 0.9200 - val_loss: 0.2850
Epoch 6/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9414 - loss: 0.2143 - val_accuracy: 0.9175 - val_loss: 0.2802
Epoch 7/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - accuracy: 0.9447 - loss: 0.2072 - val_accuracy: 0.9150 - val_loss: 0.2838
Epoch 8/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 71ms/step - accuracy: 0.9469 - loss: 0.1997 - val_acc

In [25]:
# TODO: Plot training history
# plot_training_history(history_lstm,
#                       os.path.join(OUTPUT_DIR, "part3_training_history.png"))

plot_training_history(
    history_lstm, os.path.join(OUTPUT_DIR, "part3_training_history.png")
)

In [26]:
# TODO: Evaluate on test set
# lstm_loss, lstm_acc = model_lstm.evaluate(X_test_ecg, y_test_ecg, verbose=0)
lstm_loss, lstm_acc = model_lstm.evaluate(X_test_ecg, y_test_ecg, verbose=0)  # replace

# TODO: Generate predictions and confusion matrix
# y_pred_ecg = np.argmax(model_lstm.predict(X_test_ecg, verbose=0), axis=1)
# y_true_ecg = np.argmax(y_test_ecg, axis=1)
# cm_ecg = confusion_matrix(y_true_ecg, y_pred_ecg)
y_pred_ecg = np.argmax(model_lstm.predict(X_test_ecg, verbose=0), axis=1)
y_true_ecg = np.argmax(y_test_ecg, axis=1)
cm_ecg = confusion_matrix(y_true_ecg, y_pred_ecg)

In [27]:
# Optional: visualize confusion matrix to see which heartbeat types are confused
# plot_confusion_matrix(y_true_ecg, y_pred_ecg, list(ECG_CLASSES.values()),
#                       os.path.join(OUTPUT_DIR, "part3_confusion_matrix.png"))

In [28]:
# Save results (do not modify this cell)
results_ecg = {
    "accuracy": float(lstm_acc),
    "confusion_matrix": cm_ecg.tolist(),
}
with open(os.path.join(OUTPUT_DIR, "part3_results.json"), "w") as f:
    json.dump(results_ecg, f, indent=2)

print(f"LSTM accuracy: {lstm_acc:.4f}")
print("Saved output/part3_results.json")

LSTM accuracy: 0.9250
Saved output/part3_results.json


---

## Validation

In [29]:
print("\nAll parts complete!")
print("Run 'pytest .github/tests/ -v' in your terminal to check your work.")


All parts complete!
Run 'pytest .github/tests/ -v' in your terminal to check your work.
